<img src='./img/LogoWekeo_Copernicus_RGB_0.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='20%'></img>

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./10_sentinel5p_L2_retrieve.ipynb"><< 10 - Sentinel-5P Carbon Monoxide - Retrieve</a><span style="float:right;"><a href="./20_sentinel3_OLCI_L1_retrieve.ipynb">20 - Sentinel-3 OLCI Level 1B - Retrieve >></a></span>

<div class="alert alert-block alert-warning">
<b>LOAD, BROWSE AND VISUALIZE</b></div>

# Copernicus Sentinel-5 Precursor (Sentinel-5p) - Carbon Monoxide

The subsequent example introduces you to Sentinel-5p data in general and the total column of carbon monoxide sensed by Sentinel-5p in specific. Carbon Monoxide is a good trace gas in order to monitor and track fire occurences. The example is based on the Australian bushfires that severely affected Australias East Coast end of 2019 and beginning of 2020.


#### Module outline:
* [1 - Load and browse Sentinel-5P TROPOMI data](#load_s5p)
* [2 - Create a geographical subset](#geographical_subset)
* [3 - Visualize Sentinel-5P Carbon Monoxide data](#visualize_s5p)

#### Load required libraries

In [ ]:
%matplotlib inline
import os
import xarray as xr
import numpy as np
import netCDF4 as nc

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

import subprocess
import shlex
import zipfile
import os

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh

#### Load helper functions

In [ ]:
from ipynb.fs.full.functions import visualize_pcolormesh, generate_geographical_subset

<hr>

### Decompressing Sentinel-5 Precursor (Sentinel-5p) data

The Sentinel-5 Precursor (Sentinel-5p) data could be not available on the machine. In this case there is the method to download them to [10 - Sentinel-5 Precursor Level 2 Retrieve](./10_sentinel5p_L2_retrieve.ipynb).

Also the user must extract the data before load them, if they are provided as a zip archive. If you have an already decompressed data this step is not necessary and the data can be directly loaded.

In [ ]:
#rename S3 file
download_dir_path = os.getcwd()

for item in os.listdir(download_dir_path):
    if not os.path.isdir(item):
        cmd = shlex.split('file --mime-type {0}'.format(item))
        result = subprocess.check_output(cmd)
        mime_type = result.split()[-1]
        if item.startswith('S5P_OFFL_L2_') and not item.endswith('.zip') and mime_type == b'application/zip':
            os.rename(os.path.join(download_dir_path, item), os.path.join(download_dir_path, item +".zip"))

In case you have permission issues when renaming the archive with the code above, please manually do that by right-clicking the file, then rename, then add .zip to the end of the filename.

For example:
- From: S5P_OFFL_L2__NO2____20191229T014948_20191229T033118_11445_01_010302_20191230T183957  
- To: S5P_OFFL_L2__NO2____20191229T014948_20191229T033118_11445_01_010302_20191230T183957.zip

In [ ]:
extension = ".zip"

for item in os.listdir(download_dir_path): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.join(download_dir_path, item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(download_dir_path) # extract file to dir
        zip_ref.close() # close file
        

## <a id="load_s5p"></a>Load and browse Sentinel-5P data

A Sentinel-5p file is organised in two groups: `PRODUCT` and `METADATA`. The `PRODUCT` group stores the main data fields of the product, including `latitude`, `longitude` and the variable itself. The `METADATA` group provides additional metadata items.

Sentinel-5p variables have the following dimensions:
* `scanline`: the number of measurements in the granule / along-track dimension index
* `ground_pixel`: the number of spectra in a measurement / across-track dimension index
* `time`: time reference for the data
* `corner`: pixel corner index
* `layer`: this dimension indicates the vertical grid of profile variables

Sentinel-5p data is disseminated in `netCDF`. You can load multiple `netCDF` files at once with the `open_mfdataset()` function of the xarray library. In order to load the variable as part of a Sentinel-5p data files, you have to specify the following keyword arguments: 
- `group='PRODUCT'`: to load the `PRODUCT` group
- `concat_dim='scanline'`: multiple files will be concatenated based on the scanline dimension
- `combine=nested`: combine a n-dimensional grids into one along each dimension of the grid

Let us load a Sentinel-5p data file as `xarray.Dataset` and inspect the data structure:

In [ ]:
s5p = xr.open_mfdataset('./S5P_OFFL_L2__NO2____20191229*/*.nc', concat_dim='scanline', combine='nested', group='PRODUCT')
s5p

You see that a Sentinel-5p data files contains of ten dimensions and twelve data variables:
* **Dimensions**:
  * `scanline` 
  * `ground_pixel`
  * `time`
  * `corner`
  * `polynomial_exponents`
  * `intensity_offset_polynomial_exponents`
  * `layer`
  * `vertices`
  * `latitude`
  * `longitude`


* **Data variables**:
  * `delta_time`: the offset of individual measurements within the granule, given in milliseconds
  * `time_utc`: valid time stamp of the data
  * `ga_value`: quality descriptor, varying between 0 (nodata) and 1 (full quality data)
  * `nitrogendioxide_tropospheric_column`: Vertically integrated CO column density
  * `nitrogendioxide_tropospheric_column_precision`: precision of the tropospheric vertical column of NO2
  * `nitrogendioxide_tropospheric_column_precision_kernel`: Precision of the tropospheric vertical column of NO2 when the averaging kernel is applied
  * `averaging_kernel`: averaging kernel A for in the air mass factor correction, describing the NO2 profile sensitivity of the vertical column density (dimensionless) 
  * `air_mass_factor_troposphere`: tropospheric air mass factor, computed by integrating the altitude dependent air mass factor over the atmospheric layers from  the surface up to and including the layer with the tropopause.
  * `air_mass_factor_total`: Total air mass factor, computed by integrating the altitude dependent air mass factor over the atmospheric layers from the surface to top-of-atmosphere
  * `tm5_tropopause_layer_index`: index of the highest layer in TM5 which is completely inside the troposphere, in terms of the layer coordinate
  * `tm5_constant_a`: hybrid A coefficient at the TM5 pressure levels
  * `tm5_constant_b`: hybrid B coefficient at the TM5 pressure levels
  

You can specify one variable of interest and get more detailed information about the variable. E.g. `carbonmonoxide_total_column` is the atmosphere mole content of carbon monoxide, has the unit `mol per m-2`, and has three dimensions, `time`, `scanline` and `groundpixel` respectively.

In [ ]:
s5p_co = s5p['nitrogendioxide_tropospheric_column']
s5p_co

You can do this for the available variables, but also for the dimensions latitude and longitude.

In [ ]:
print('Latitude')
print(s5p_co.latitude)

print('Longitude')
print(s5p_co.longitude)

<br>

You can retrieve the array values of the variable with squared bracket: `[:,:,:]`. One single time step can be selected by specifying one value of the time dimension, e.g. `[0,:,:]`.

In [ ]:
s5p_co_2912 = s5p_co[0,:,:]
s5p_co_2912

The attributes of the data array hold the entry `multiplication_factor_to_convert_to_molecules_percm2`, which is a conversion factor that has to be applied to convert the data from `mol per m2` to `molecules per cm2`.


In [ ]:
conversion_factor = s5p_co_2912.multiplication_factor_to_convert_to_molecules_percm2
conversion_factor

Additionally, you can save the attribute `longname`, which you can make use of when visualizing the data.

In [ ]:
longname = s5p_co.long_name
longname

## <a id='geographical_subset'></a>Create  a geographical subset

You can zoom into a region by specifying a `bounding box` of interest. Let's set the extent to southeast Australia with the following bounding box information:

In [ ]:
lonmin=135.39739634141876
lonmax=162.5751370327434
latmin=-44.52932633885725
latmax=-13.045119366579158

You can use the function [generate_geographical_subset()](./functions.ipynb#generate_geographical_subset) to subset an xarray DataArray based on a given bounding box.

In [ ]:
s5p_co_subset = generate_geographical_subset(s5p_co_2912, latmin, latmax, lonmin, lonmax)
s5p_co_subset

<br>

## <a id="plotting_s5p"></a>Plotting example - Sentinel-5P data

You can plot data arrays of type `numpy` with matplotlib's `pcolormesh` function. In combination with the library [cartopy](https://scitools.org.uk/cartopy/docs/latest/), you can produce high-quality maps. 

In order to make it easier to visualize the Carbon Monoxide values, we apply the conversion factor to the DataArray. This converts the Carbon Monoxide values from mol per m<sup>2</sup> to molecules per cm<sup>2</sup>.

In [ ]:
s5p_co_converted = s5p_co_subset*conversion_factor
s5p_co_converted

For visualization, you can use the function [visualize_pcolormesh](./functions#visualize_pcolormesh) to visualize the data. The following kwargs have to be defined:
* `xarray DataArray`
* `longitude`
* `latitude`
* `projection`
* `color palette`
* `unit`
* `longname`
* `vmin`, `vmax`
* `extent (lonmin, lonmax, latmin, latmax)`
* `log`
* `set_global`

Now, let us apply the [visualize_pcolormesh](./functions#visualize_pcolormesh) function and visualize the vertically integrated carbon monoxide column sensored from the Sentinel-5P satellite on 29 December 2019.

Note: Multiplying the DataArray values with 1e-18 improves the readibility of the map legend.

In [ ]:
visualize_pcolormesh(s5p_co_converted*1e-18, 
                     s5p_co_converted.longitude, 
                     s5p_co_converted.latitude, 
                     ccrs.PlateCarree(), 
                     'viridis', 
                     '*1e-18 molecules per cm2', 
                     longname, 
                     0, 8, 
                     lonmin, lonmax, latmin, latmax, 
                     log=False, 
                     set_global=False)


<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./10_sentinel5p_L2_retrieve.ipynb"><< 10 - Sentinel-5P Carbon Monoxide - Retrieve</a><span style="float:right;"><a href="./20_sentinel3_OLCI_L1_retrieve.ipynb">20 - Sentinel-3 OLCI Level 1B - Retrieve >></a></span>

<hr>

<img src='./img/all_partners_wekeo.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='100%'></img>